<a href="https://colab.research.google.com/github/vmadalasa/EVA4S5/blob/master/Madhu_session5Model6_with_batchsize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MODEL OBSERVATIONS



Target:
In this trial, we want to tweak the model gradients by changing batch size

Results:
Parameters: 9468



Analysis:
Batch size of 64 leads to better results



Second Skeleton model with batch norm and without regularisation (dropouts)
Input size 28X28X1; output_size = 26X26X12; RF 3X3 - Convinput - with batchnorm and dropout

Input size 26X26X12; output_size = 24X24X12; RF 5X5 - convblock1A- with batchnorm and dropout Input size 24X24X12; output_size = 22X22X20; RF 7X7 - convblock1B- with batchnorm and dropout

Now Max pool to get this to Input size 22X22X20; output_size = 11X11X20; RF 14X14 - pool1 Input size 11X11X20 output_size = 11X11X12; RF 14X14 - tranblock1

Convblock 2 Input size 11X11X12; output_size = 9X9X12; RF 16X16 - convblock2A- with batchnorm and dropout Input size 9X9X12; output_size = 7X7X16; RF 18X18 convblock2B- with batchnorm and dropout Input size 7X7X16; output_size = 5X5X16; RF 20X20 convblock2C- with batchnorm and dropout

We do self.gap layer at kernel size 5 - gap finally transblock 1X1 for Input size 1X1X16; output_size = 1X1X10; RF 20X20 - convblockEnd WITHOUT batch norm here

BATCH SIZE = 64


In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:


dropout_value2 = 0.1
dropout_value1 = 0.1
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()


        # Input Block
        self.convinput = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=12, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Dropout(dropout_value2)
        ) # Input size 28X28X1; output_size = 26X26X12; RF 3X3

       # CONVOLUTION BLOCK 1
        self.convblock1A = nn.Sequential(
            nn.Conv2d(in_channels=12, out_channels=12, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Dropout(dropout_value2)
        ) # Input size 26X26X12; output_size = 24X24X12; RF 5X5

        self.convblock1B = nn.Sequential(
            nn.Conv2d(in_channels=12, out_channels=20, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(20),
            nn.Dropout(dropout_value2)
        ) # Input size 24X24X12; output_size = 22X22X12; RF 7X7

        
           ##----------------------------##     
        #self.convblock1C = nn.Sequential(
        #    nn.Conv2d(in_channels=12, out_channels=12, kernel_size=(3, 3), padding=1, bias=False),
        #    nn.ReLU(),
        #    nn.BatchNorm2d(12),
        #    nn.Dropout(dropout_value2)
        #) # Input size 22X22X12; output_size = 22X22X12; RF 9X9

         ##----------------------------##

        # TRANSITION BLOCK 1

        self.pool1 = nn.MaxPool2d(2, 2) # # Input size 22X22X12; output_size = 11X11X12; RF 14X14
        
        self.tranblock1 = nn.Sequential(
           nn.Conv2d(in_channels=20, out_channels=12, kernel_size=(1, 1), padding=0, bias=False),
        ) # # Input size 11X11X16; output_size = 11X11X12; RF 14X14



        ##----------------------------##
        #self.poolAlt1 = nn.Sequential(
        #   nn.Conv2d(in_channels=12, out_channels=16, kernel_size=(3, 3), stride=2,padding=1, bias=False),
        #  nn.ReLU(),            
        # nn.BatchNorm2d(16),
        #nn.Dropout(dropout_value)
        #) # # Input size 22X22X12; output_size = 11X11X16; RF 18X18

       ##---------------------------##


      # CONVOLUTION BLOCK 2
        self.convblock2A = nn.Sequential(
            nn.Conv2d(in_channels=12, out_channels=12, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Dropout(dropout_value)
        ) # Input size 11X11X12; output_size = 9X9X12; RF 16X16

 
        self.convblock2B = nn.Sequential(
            nn.Conv2d(in_channels=12, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # # Input size 9X9X12; output_size = 7X7X16; RF 18X18

        self.convblock2C = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # # Input size 7X7X16; output_size = 5X5X16; RF 20X20


      # OUTPUT BLOCK
        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=5)
        ) # output_size = 1

        self.convblockEnd = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
            # nn.BatchNorm2d(10),
            # nn.ReLU(),
            # nn.Dropout(dropout_value) Input size 1X1X16; output_size = 1X1X10; RF 20X20

        ) 

        self.dropout = nn.Dropout(dropout_value)

    def forward(self, x):
        x = self.convinput(x)

        x = self.convblock1A(x)
        x = self.convblock1B(x)
        #x = self.convblock1C(x)
        
        x = self.pool1(x)
        #x = self.poolAlt1(x)
        x = self.tranblock1(x)

        x = self.convblock2A(x)
        x = self.convblock2B(x)
        x = self.convblock2C(x)

        x = self.gap(x)        
       
        x = self.convblockEnd(x)
        #print(x.shape)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)
        
        
     

In [0]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 12, 26, 26]             108
              ReLU-2           [-1, 12, 26, 26]               0
       BatchNorm2d-3           [-1, 12, 26, 26]              24
           Dropout-4           [-1, 12, 26, 26]               0
            Conv2d-5           [-1, 12, 24, 24]           1,296
              ReLU-6           [-1, 12, 24, 24]               0
       BatchNorm2d-7           [-1, 12, 24, 24]              24
           Dropout-8           [-1, 12, 24, 24]               0
            Conv2d-9           [-1, 20, 22, 22]           2,160
             ReLU-10           [-1, 20, 22, 22]               0
      BatchNorm2d-11           [-1, 20, 22, 22]              40
          Dropout-12           [-1, 20, 22, 22]               0
        MaxPool2d-13           [-1, 20, 11, 11]               0
           Conv2d-14           [-1, 12,

In [0]:


torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                         transforms.RandomRotation((-9.0, 9.0), fill=(1,)),                    
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)




In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
   

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#optimizer = optim.Adam(model.parameters(), lr=0.001,betas=(0.9,0.99))

for epoch in range(1, 16):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    print('Epoch', epoch, ' done')

  0%|          | 0/1875 [00:00<?, ?it/s]

EPOCH: 1


loss=0.06964191794395447 batch_id=1874 Accuracy=93.16: 100%|██████████| 1875/1875 [00:41<00:00, 45.40it/s]
  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0561, Accuracy: 9830/10000 (98.30%)

Epoch 1  done
EPOCH: 2


loss=0.06856218725442886 batch_id=1874 Accuracy=97.42: 100%|██████████| 1875/1875 [00:40<00:00, 45.77it/s]
  0%|          | 0/1875 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0205, Accuracy: 9944/10000 (99.44%)

Epoch 13  done
EPOCH: 14


loss=0.01454995572566986 batch_id=1874 Accuracy=98.72: 100%|██████████| 1875/1875 [00:39<00:00, 47.30it/s]
  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0236, Accuracy: 9928/10000 (99.28%)

Epoch 14  done
EPOCH: 15


loss=0.002143383026123047 batch_id=1874 Accuracy=98.74: 100%|██████████| 1875/1875 [00:39<00:00, 46.89it/s]



Test set: Average loss: 0.0204, Accuracy: 9945/10000 (99.45%)

Epoch 15  done
